In [1]:
import numpy as np
import pandas as pd

In [2]:
PATH2 = './../TlkAgg2/'
PATH5 = './../TlkAgg5/'
PATH = PATH5

In [3]:
data = pd.read_csv(PATH + 'crowd_labels.tsv', sep='\t', header=None)
data.columns = ['worker_id', 'task_id', 'verdict']
data = data.set_index('task_id')
if PATH == PATH5:
    data['verdict'] = data['verdict'] - 1
data

,worker_id,verdict
task_id,,
t186849,w360,3
t174611,w43,4
t198665,w1040,2
t63171,w618,4
t141574,w98,4
...,...,...
t347029,w883,3
t315605,w328,4
t26047,w468,2


In [4]:
z_j = data.groupby(['task_id','verdict'])['worker_id'].count()
z_j = z_j.unstack(level = 1).fillna(0)
z_j['sum'] = z_j.sum(axis = 1)
z_j = z_j.apply(lambda x : x/z_j['sum'])
del z_j['sum']
z_j['z_j'] = z_j.apply(lambda x : [x.values.tolist()], axis = 1)
z_j = z_j[['z_j']]
z_j

verdict,z_j
task_id,
t0,"[[0.0, 0.0, 1.0, 0.0, 0.0]]"
t1,"[[0.0, 0.0, 0.0, 0.3333333333333333, 0.6666666..."
t10,"[[0.0, 0.0, 0.0, 0.0, 1.0]]"
t100,"[[0.0, 0.0, 0.0, 0.3333333333333333, 0.6666666..."
t1000,"[[0.0, 0.0, 0.0, 0.0, 1.0]]"
...,...
t99995,"[[0.0, 0.0, 0.3333333333333333, 0.666666666666..."
t99996,"[[0.0, 0.0, 1.0, 0.0, 0.0]]"
t99997,"[[0.0, 0.0, 0.0, 0.0, 1.0]]"


In [5]:
verdicts = data['verdict'].sort_values().unique()

In [6]:
def h1_m_step(i):
    l = np.zeros([len(verdicts), 1])
    l[i][0] = 1
    return l

In [7]:
def h2_m_step(x):
    m = x.copy()
    l = np.sum(a = m, axis = 0)
    for i in range(len(m)):
        m[i] = np.divide(m[i], l, out = np.zeros_like(m[i]), where = l!=0)
    return m

In [8]:
def m_step(data, z_j):
    e_w = data.merge(z_j, left_index = True, right_index = True, how = 'left')
    e_w['l'] = e_w['verdict'].apply(h1_m_step)
    e_w['e_w'] = e_w.apply(lambda x : np.matmul(x[3], x[2]), axis = 1)
    e_w = e_w.groupby('worker_id')['e_w'].apply(np.sum)
    e_w = e_w.apply(h2_m_step).to_frame()
    p = np.sum(z_j['z_j'].tolist(), axis = 0)
    p = p / np.sum(np.sum((p)))
    return e_w, p

In [9]:
def h1_e_step(i):
    l = np.zeros([1, len(verdicts)])
    l[0][i] = 1
    return l

In [10]:
def h2_e_step(x):
    m = x.copy()
    m = m / np.sum(m)
    return m

In [11]:
def e_step(data, e_w, p):
    z_j = data.merge(e_w, left_on = 'worker_id', right_index = True, how = 'left')
    z_j['l'] = z_j['verdict'].apply(h1_e_step)
    z_j['z_j'] = z_j.apply(lambda x : np.matmul(x[3], x[2]), axis = 1)
    z_j = z_j.groupby('task_id')['z_j'].apply(np.prod).apply(lambda x : np.multiply(x, p))
    z_j = z_j.apply(h2_e_step).to_frame()
    return z_j

In [12]:
for k in range(10):
    e_w, p = m_step(data, z_j)
    z_j = e_step(data, e_w, p)
z_j

,z_j
task_id,
t0,"[[5.923670427222245e-06, 0.04725264905446727, ..."
t1,"[[7.447369653788224e-10, 4.943942909695502e-06..."
t10,"[[1.2994441768410283e-06, 3.981115050695818e-0..."
t100,"[[0.00011244394888041456, 4.2423345401839275e-..."
t1000,"[[4.059655048529988e-05, 3.170275466446619e-06..."
...,...
t99995,"[[0.0001968184556749363, 0.006766559968624152,..."
t99996,"[[1.796743753978418e-07, 0.04641869373932869, ..."
t99997,"[[1.4189997042548532e-08, 1.9869064986301656e-..."


In [13]:
if PATH == PATH2:
    z_j_result = z_j['z_j'].apply(np.argmax).to_frame()
else:
    z_j_result = z_j['z_j'].apply(lambda x : 1 + np.argmax(x)).to_frame()
z_j_result

,z_j
task_id,
t0,3
t1,5
t10,5
t100,5
t1000,5
...,...
t99995,4
t99996,3
t99997,5


In [14]:
z_j_result.to_csv(PATH + 'ds.tsv', sep = '\t', header=None)